In [1]:
# going to try to create my own word2vec embedding using gensim --> doing it here to save disk space
# this version will use the text8 corpus: http://mattmahoney.net/dc/text8.zip

# get gensim common_text corpus and word2vec model
from gensim.test.utils import common_texts, get_tmpfile
from gensim.models import Word2Vec
from gensim.models.word2vec import Text8Corpus
import numpy as np
import tensorflow as tf
from keras.layers import Embedding
from keras.engine import Input
from keras.models import Model
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from keras.layers import LSTM
import keras

/Users/chloeloughridge/anaconda/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.5
  return f(*args, **kwds)
Using TensorFlow backend.


In [2]:
# train the word2vec thing on this and create vectors for each word in vocab
# inspiration: https://radimrehurek.com/gensim/models/word2vec.html
path = get_tmpfile("word2vec.model") # what does this do and do I need it?

sentences = Text8Corpus("/Users/chloeloughridge/TextGeneration/text8")
#model.build_vocab(sentences)  # prepare the model vocabulary --> do I actually need this? i think not, but oh well
model = Word2Vec(sentences, size=100, window=5, min_count=1, workers=4) # what are workers?
model.save("word2vec.model") # what is the advantage to doing this?

print(model['debod'])

[-0.03208853  0.0073209   0.00900712 -0.02446695 -0.00163148  0.01236735
 -0.00525934 -0.00280406 -0.01553019  0.02511849 -0.01539957  0.00375687
  0.02261087 -0.00594487 -0.01470008  0.02310156  0.0162046   0.02720891
  0.02658098  0.01824113 -0.00542357  0.01577971 -0.00168987 -0.00442371
  0.02118422 -0.02085195 -0.01498204 -0.03048212 -0.00604855 -0.01178851
  0.01487743  0.02653154  0.00581537  0.02968577  0.0028181  -0.04032163
  0.01518094  0.00141743 -0.00661938  0.00267021  0.01460859 -0.00352043
  0.02159883 -0.00027704  0.01760519  0.01320309 -0.0067158  -0.00229618
 -0.00172614  0.01947322  0.02285793  0.03966976  0.00404166  0.00447733
  0.00125471  0.0077698  -0.00174025 -0.00056025 -0.03043101 -0.02269287
  0.01442108 -0.02012351  0.01134761  0.0141385   0.01430881  0.00147922
 -0.01534944  0.0070199  -0.01092546 -0.03044373  0.00966285  0.00321195
  0.00963955 -0.01097647  0.02345185 -0.00660561 -0.00932538  0.01753954
 -0.01894061  0.01131629 -0.00976695  0.02295437 -0

/Users/chloeloughridge/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:10: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


In [3]:
# how to you get to the sentences within sentences? 
# how to access the inards of a Text8Corpus object??
print(sentences)
counter = 0
max_length = 0
for sentence in sentences:
    counter = counter + 1
    counter_Two = 0
    for word in sentence:
        counter_Two += 1
    if counter_Two > max_length:
        max_length = counter_Two
print(counter)
print("words per sentence: {}".format(counter_Two))
num_sentences = counter
print(max_length)

MAX_SEQUENCE_LENGTH = max_length

1701
words per sentence: 5207
10000


In [4]:
print(model['debod'].shape)

(100,)


/Users/chloeloughridge/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if __name__ == '__main__':


In [5]:
# create the embedding matrix out of these word vectors: http://adventuresinmachinelearning.com/gensim-word2vec-tutorial/
# the final result should be a matrix with shape (vocab_length, vector_dims)

# convert the wv word vectors into a numpy matrix that is suitable for insertion
# into our TensorFlow and Keras models
vector_dim = 100
embedding_matrix = np.zeros((len(model.wv.vocab), vector_dim))
for i in range(len(model.wv.vocab)):
    embedding_vector = model.wv[model.wv.index2word[i]]
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [6]:
#MAX_SEQUENCE_LENGTH = 1000
sentence = "the big apple"
word = sentence[1]
# this is my word_to_index map
print(model.wv.vocab[word].index)
# this is my word_to_vec map
print(model['bug'])

256
[ 2.4928477e-02  7.4395560e-02  1.4572303e-01 -1.8398167e-01
 -4.6665215e-01  6.4402813e-01 -2.5188529e-01  9.6568957e-02
  4.6680573e-02 -7.2990179e-01 -1.1497512e+00 -1.1323198e-01
  2.2131431e-01  1.1476829e+00 -9.4561452e-01 -4.3233284e-01
 -2.4194548e-01  1.3786192e-01  4.4658485e-01 -5.1966548e-01
  5.3422958e-01  4.1615844e-01  6.0259962e-01 -1.9902055e-01
 -6.8172431e-01 -6.4788491e-01 -3.9962593e-01  2.8245482e-01
 -4.1540027e-01 -4.9640048e-01 -3.3802402e-01 -1.3220663e-01
  1.2126574e-01 -6.8992680e-01  3.8853800e-01 -3.1930414e-01
  7.4009663e-01 -5.1237494e-02 -2.2016181e-01 -3.1909007e-01
 -3.0921206e-01 -3.0908048e-01  2.1386090e-01 -7.2627753e-01
  2.8904381e-01 -8.2442200e-01 -1.6690338e-01  2.7233225e-01
 -7.3778182e-01  3.2042569e-01  9.9283761e-01  9.5139466e-02
 -1.1157016e+00 -2.6989222e-01  6.4047486e-01  8.2368925e-02
  9.7128052e-01 -4.7904690e-04 -1.2479106e+00 -9.5203042e-01
  6.4324236e-01 -6.8074816e-01  1.2045791e+00 -8.5456014e-01
 -1.1465035e-01  5.1

/Users/chloeloughridge/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:7: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


In [7]:
# testing iteration through word to index mapping
word_to_index = model.wv.vocab
#for word in word_to_index:
    #print(word_to_index[word].index)

In [8]:
# figuring out word_to_vec_map and word_to_index map

In [9]:
# function for creating embedding layer inspired by Ng course
def pretrained_embedding_layer(word_to_vec_map, word_to_index):
    
    vocab_len = len(word_to_index) + 1                  # adding 1 to fit Keras embedding (requirement)
    emb_dim = word_to_vec_map["cucumber"].shape[0]      # define dimensionality of your GloVe word vectors (= 50)
    
    ### START CODE HERE ###
    # Initialize the embedding matrix as a numpy array of zeros of shape (vocab_len, dimensions of word vectors = emb_dim)
    emb_matrix = np.zeros([vocab_len, emb_dim])
    
    # Set each row "index" of the embedding matrix to be the word vector representation of the "index"th word of the vocabulary
    for word in word_to_index:
        #print(word.index)
        index = word_to_index[word].index
        emb_matrix[index] = word_to_vec_map[word]

    # Define Keras embedding layer with the correct output/input sizes, make it trainable. Use Embedding(...). Make sure to set trainable=False. 
    embedding_layer = Embedding(input_dim=vocab_len, output_dim=emb_dim, trainable=False)
    ### END CODE HERE ###

    # Build the embedding layer, it is required before setting the weights of the embedding layer. Do not modify the "None".
    embedding_layer.build((None,))
    
    # Set the weights of the embedding layer to the embedding matrix. Your layer is now pretrained.
    embedding_layer.set_weights([emb_matrix])
    
    return embedding_layer

In [10]:
# building the model

input_shape = (MAX_SEQUENCE_LENGTH,)
sequence_input = Input(shape=input_shape)
# set up the embedding layer
embedding_layer = pretrained_embedding_layer(model, model.wv.vocab)
# send the input through the embedding layer 
embeddings = embedding_layer(sequence_input) 
print(embeddings.shape)
# now for the rest of the model
X = LSTM(28)(embeddings)
#X = Flatten()(X) --> we actually don't need the flatten layer
X = Dense(MAX_SEQUENCE_LENGTH, activation='softmax')(X)

# instantiate model
LSTM_model = Model(inputs=sequence_input, outputs=X)


/Users/chloeloughridge/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
/Users/chloeloughridge/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:15: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


(?, 10000, 100)


In [12]:
# compile the model
LSTM_model.compile(loss='binary_crossentropy',
              optimizer='adam', # or is rmsprop better??
              metrics=['binary_accuracy'])

In [13]:
# get the input data in shape --> I think we need to convert each word to one-hot representation
for sentence in sentences:
    # initialize sentence array of shape [max_seq_length, vocab_length]
    sent_array = np.zeros([MAX_SEQUENCE_LENGTH, len(model.wv.vocab)])
    counter = 0
    for word in sentence:
        #print(counterTwo)
        # get the index of that word and use to_categorical function to get one-hot
        word_oh = keras.utils.to_categorical(model.wv.vocab[word].index, len(model.wv.vocab))
        # set the sentence array at the index “counterTwo” to the one-hot output of previous line
        sent_array[counter] = word_oh
        counter = counter + 1

print(sent_array)

KeyboardInterrupt: 

In [17]:
for j in range(len(sent_array)):
    one = False
    for i in range(len(sent_array[0])):
        if sent_array[0][i] == 1:
            one = True
    if not one:
        print("stopped at row {}".format(j))

print(sent_array[0])

KeyboardInterrupt: 

In [ ]:
# train the model

In [ ]:
# sample the model

# choose a random starting word and then see where it goes from there?
